# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Next we used an indepandent train.py file to first get the data extracted from https://gist.githubusercontent.com/Nwaneto/0d1477bd10c92f8b16ab19306d21a17f/raw/0af3078c0d174e26039ab31525487ceaceda77b0/parkinson-classification-data.csv , then define the parameters (C , max_iter), then clean the data (by removing the name of the personne and defining the dependent and independent variable) and finally set the training and the testing data

Train.py
from sklearn.linear_model import LogisticRegression

import argparse

import os

import numpy as np

from sklearn.metrics import mean_squared_error

import joblib

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

import pandas as pd

from azureml.core.run import Run

from azureml.data.dataset_factory import TabularDatasetFactory

def clean_data(data):

# Clean the data x_df = data.to_pandas_dataframe().dropna() x_df.drop("name", inplace=True, axis=1) y_df = x_df.pop("status")

return x_df, y_df

def main(): # Add arguments to the script parser = argparse.ArgumentParser()

parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization") parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

args = parser.parse_args()

# Create TabularDataset using TabularDatasetFactory # Data is located at:

path_file="https://gist.githubusercontent.com/Nwaneto/0d1477bd10c92f8b16ab19306d21a17f/raw/0af3078c0d174e26039ab31525487ceaceda77b0/parkinson-classification-data.csv"

`ds =TabularDatasetFactory.from_delimited_files(path=path_file)`

x, y = clean_data(ds)

`# Split data into train and test sets.`

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

run = Run.get_context()

run.log("Regularization Strength:", np.float(args.C)) run.log("Max iterations:", np.int(args.max_iter))

model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

accuracy = model.score(x_test, y_test) run.log("Accuracy", np.float(accuracy))

if __name__ == '__main__': main()



In [2]:
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification-HDrive'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-137760
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-137760


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The algorithm we choose for this classification problem, is LogisticRegression because we are trying to predict if a patient will have the parkinson disease based on a range of biomedical voice measurements (yes or no) which means two outcomes.

And To improve the model we optimize the hyperparameters using Azure Machine Learning's tuning capabilities Hyperdrive

First of all, we define the hyperparameter space to sweep over. which means tuning the C and max_iter parameters. In this step, we use the random sampling RandomParameterSampling to try different configuration sets of hyperparameters to maximize the primary metric to make the tuning more specific

Then we define the termination Policy for every run using BanditPolicy based on a slack factor equal to 0.01 as criteria for evaluation to conserves resources by terminating runs that are poorly performing and anssure that every run will give better result than the one before

Once completed we create the SKLearn estimator

An finally we define the hyperdrive configuration where we set 20 as the maximum of iteration (why because we don't have a lot of data) and used the element defined above before submiting the experiment

In [3]:
amlcompute_cluster_name = "cpu-clusters"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [4]:
#Create the different params that  will be using during training
param_sampling =RandomParameterSampling( {
    "--C":  choice(0.1, 0.2, 0.3, 0.4, 0.5),
    "--max_iter":  choice(100, 150, 200, 250, 300)
    }
)

#Create an early termination policy.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.01)

#Create the estimator and the hyperdrive
estimator =  SKLearn(source_directory='./', 
                entry_script='train.py', compute_target=aml_compute)


hyperdrive_run_config =HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                    primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    policy=early_termination_policy,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    estimator=estimator
                                   )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [ ]:
#Submit the experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

In [ ]:
#Save and register the best model
model = best_run.register_model(model_name='Parkinson_detection', model_path='./')